In [47]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [48]:
train = pd.read_csv('Train_UWu5bXk.csv')
test = pd.read_csv('Test_u94Q5KV.csv')
train['source']='train'
test['source']='test'
data = pd.concat([train, test],ignore_index=True)
print (train.shape, test.shape, data.shape)

(8523, 13) (5681, 12) (14204, 13)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [49]:
data.head(3)

,Item_Fat_Content,Item_Identifier,Item_MRP,Item_Outlet_Sales,Item_Type,Item_Visibility,Item_Weight,Outlet_Establishment_Year,Outlet_Identifier,Outlet_Location_Type,Outlet_Size,Outlet_Type,source
0,Low Fat,FDA15,249.8092,3735.1380,Dairy,0.016047,9.30,1999,OUT049,Tier 1,Medium,Supermarket Type1,train
1,Regular,DRC01,48.2692,443.4228,Soft Drinks,0.019278,5.92,2009,OUT018,Tier 3,Medium,Supermarket Type2,train
2,Low Fat,FDN15,141.6180,2097.2700,Meat,0.016760,17.50,1999,OUT049,Tier 1,Medium,Supermarket Type1,train


In [50]:
data.apply(lambda x: sum(x.isnull()))

Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Outlet_Sales            5681
Item_Type                       0
Item_Visibility                 0
Item_Weight                  2439
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                  4016
Outlet_Type                     0
source                          0
dtype: int64

In [51]:
categorical_columns = [x for x in data.dtypes.index if data.dtypes[x]=='object']
categorical_columns = [x for x in categorical_columns if x not in ['Item_Identifier','Outlet_Identifier','source']]
for col in categorical_columns:
    print ('\nFrequency of Categories for varible %s'%col)
    print (data[col].value_counts())


Frequency of Categories for varible Item_Fat_Content
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Frequency of Categories for varible Item_Type
Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

Frequency of Categories for varible Outlet_Location_Type
Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

Frequency of Categories for varible Outlet_Size
Medium    4655
Small     3980
High      1553
Name: Outlet_Size, dtype: int64

F

In [52]:
data.Item_Weight = data.Item_Weight.fillna(data.Item_Weight.mean())

In [53]:
from scipy.stats import mode

outlet_size_mode = data.pivot_table(values='Outlet_Size', columns='Outlet_Type',aggfunc=(lambda x:mode(x).mode[0]) )

miss_bool = data['Outlet_Size'].isnull() 

data.loc[miss_bool,'Outlet_Size'] = data.loc[miss_bool,'Outlet_Type'].apply(lambda x: outlet_size_mode[x])
print (sum(data['Outlet_Size'].isnull()))

0


In [54]:
data.Item_Visibility=data.Item_Visibility.replace(0.0, data.Item_Visibility.mean())

In [55]:
#Get the first two characters of ID:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])
#Rename them to more intuitive categories:
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})
data['Item_Type_Combined'].value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Type_Combined, dtype: int64

In [56]:
data['Outlet_Years'] = 2013 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

count    14204.000000
mean        15.169319
std          8.371664
min          4.000000
25%          9.000000
50%         14.000000
75%         26.000000
max         28.000000
Name: Outlet_Years, dtype: float64

In [57]:
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
print (data['Item_Fat_Content'].value_counts())

Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64


In [58]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Outlet'] = le.fit_transform(data['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
le = LabelEncoder()
for i in var_mod:
    data[i] = le.fit_transform(data[i])

In [59]:
data = pd.get_dummies(data, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                              'Item_Type_Combined','Outlet'])

In [60]:
data.dtypes

Item_Identifier               object
Item_MRP                     float64
Item_Outlet_Sales            float64
Item_Type                     object
Item_Visibility              float64
Item_Weight                  float64
Outlet_Establishment_Year      int64
Outlet_Identifier             object
source                        object
Outlet_Years                   int64
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
Item_Type_Combined_0           uint8
Item_Type_Combined_1           uint8
Item_Type_Combined_2           uint8
Outlet_0                       uint8
Outlet_1                       uint8
O

In [61]:
#Drop the columns which have been converted to different types:
data.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)

#Divide into test and train:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

#Drop unnecessary columns:
test.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)
train.drop(['source'],axis=1,inplace=True)

#Export files as modified versions:
train.to_csv("train_modified.csv",index=False)
test.to_csv("test_modified.csv",index=False)

In [62]:
train = pd.read_csv('train_modified.csv')
test = pd.read_csv('test_modified.csv')

train = train.drop(['Item_Identifier','Outlet_Identifier'],axis=1)

Y_train = train.pop('Item_Outlet_Sales')
X_train = train.iloc[:,:]

test = test.drop(['Item_Identifier','Outlet_Identifier'],axis=1)
X_test = test.iloc[:,:]


In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

In [65]:
Y_pred = regressor.predict(X_test)

In [66]:
for item in Y_pred:
    print (item)

1611.1630722740604
1233.1861950754226
901.3052812342156
2447.00063219151
5726.511574424637
1871.5092156522917
519.6985915810745
2192.948116774946
1513.8413960884538
3600.821452479374
1875.0005013691373
1285.436366663584
592.4815499303899
2278.579279841315
884.7153898689678
2343.124344278129
3173.6974144826177
3122.313211338009
3186.883352231097
537.332640961412
2881.380919063768
3752.467739051235
789.9203207075825
263.44589527514074
3495.348103440641
647.8514391413155
881.8388745224173
1965.0722442585145
3660.910214470659
2344.6029172837507
84.85634319819484
2599.065550502266
334.21808001991934
5750.794605872348
1873.9022928315314
5864.98833843644
2247.051947027266
225.54187251474892
3197.8733434707115
2209.8069741641875
1529.7256232854497
164.9092298376962
2238.093262805225
1858.6688452453525
919.6149376945139
1795.395593358486
58.862841990329784
2873.2604043412225
3674.5717312738598
1991.581886543039
659.5940194056966
1672.2322246364392
3408.180889807527
1984.3164105351789
4591.54600

350.07659791835874
2210.1683671912956
352.22122649001506
1745.434452218673
476.3250394377923
3530.7811425712807
610.6889646414885
3805.613596692124
606.418306805338
3340.9254329715163
2531.6649875770017
4538.1699384061485
3235.399076105517
1348.0126814863086
2115.3164030978637
846.8553754219997
4161.914466971554
3708.8498364935763
2001.3773733230078
453.2581817816524
2214.844742670136
3780.039291931934
1712.8622269375007
1858.8169321031196
1845.12414886532
2512.1471237130604
3064.405861802191
2808.578422259317
1915.779606406604
1362.9321212426355
2403.1310014885416
817.1226003210265
601.899776081901
2705.7096314784703
3327.1726214559053
1984.260704205415
1912.5700638754904
2737.0447227320383
3192.579931222129
2747.5976569740874
601.4666884663668
2294.6154760363884
2436.5353918349765
2734.4877063235595
1156.9003746500803
1233.739118700355
923.0546992670528
3798.9214017734284
3267.7070881359564
5794.676712863039
256.99705931025056
226.53216783524167
742.4686724562856
793.0241422973226
17

1275.4779140633857
3659.123084796629
1499.8121088088167
1064.9041575167134
2330.0261494355104
2744.598974995561
2369.9314028777344
1935.646388731438
293.16494201941777
2994.422345919036
3983.5799001154296
844.1898640448757
610.3785556948164
1719.3924008374622
829.4878821815023
4247.983545701194
2313.117467156494
429.16416486463567
2987.4280199307045
2887.9539233505325
1386.202922289163
881.817230112236
2188.9789378100254
92.08683441937437
1089.5880481957522
3474.6183971930545
2844.5070066114267
3310.7477519282274
146.86587788015495
1267.5337528440177
2103.6295706431247
3843.221598712412
3569.4040626538326
4067.2566112926866
363.21199460498264
2967.5203592599273
222.32428522460688
371.82161537975213
5215.29143439253
4422.360349497526
1564.907701118102
3002.7715031213966
2698.8448664037605
840.5842323947464
2869.8024301309624
1889.0215129410137
2014.1757304113582
1068.8778108396718
1304.4942765889605
1390.8627182149035
2060.3237759072535
1868.7005724840278
522.7704478649174
923.225909163

5247.97785567433
1470.2385840428678
883.5068492509502
2299.7690377518115
1344.613046687511
2717.1308177251
1432.922149267741
3267.9578547394353
3056.338364350905
3397.849943543831
2145.1238611849453
2629.102894719517
2493.00861234253
918.1033866740788
1812.6334769391162
1529.2258261244456
3781.5458455188564
6247.416720861209
2704.769071366233
2674.412079432417
242.36644383264732
3004.7914911875523
2796.828422856977
238.00850607435234
525.0011553795449
1156.5465201506015
635.9072600517235
431.58634137756235
3566.6696610589283
2869.708262789951
2466.9881138042642
4204.461077192201
1263.8292700634493
1713.6396237783706
1808.7933256230715
335.78460542888195
2602.477876503509
3094.3453445915115
1796.3571446120059
252.45303698374536
1568.8173466700575
4053.8173547788115
1530.2049866791313
4240.035431647606
2355.7576636225285
2496.315953004272
3261.8376740178833
3064.5606085149766
5076.738535752254
1410.2270170932902
2875.9804505588886
1059.5011893640913
718.079688606793
3310.6996899215824
41

4636.636171966192
193.34084796968526
3819.8517452244296
2908.527268825762
767.466345978311
205.90207212353036
4976.393240497595
403.4871714609815
2240.063893349193
779.9101647497499
3929.272742920718
3817.460999938357
1432.0145927127603
4141.448434126004
2095.6682613076277
2332.126011712192
5419.328513033305
4427.063489424842
2736.106072118437
2113.012342796573
737.4910032248243
137.9779337146486
3878.8839273938443
5127.150285554012
2159.9204601106126
2565.3583141521217
2790.960053107781
2934.764878732236
1876.8725496866857
1187.245234015952
1632.7906902871948
4101.604717322853
2578.2673652802246
1452.7565311117678
1198.792119710323
2671.0789404618795
2204.169659024685
1346.188781680293
3077.9258419251487
2298.445207820687
4894.288565479008
2183.9603824346796
2471.542362689161
2205.0838839229245
756.3500739213289
2951.8845941540094
544.7770470933003
1983.0562630827942
3323.574246237185
488.10808522671255
668.0113289331388
4996.993122993072
4951.126414548459
2058.4565250012333
939.84270

In [67]:
sample_file = pd.read_csv('SampleSubmission_TmnO39y.csv')
sample_file

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1686.592455
1,FDW14,OUT017,1377.134667
2,NCN55,OUT010,660.745973
3,FDQ58,OUT017,2148.875553
4,FDY38,OUT027,6186.595442
5,FDH56,OUT046,1901.137425
6,FDL48,OUT018,677.033862
7,FDC48,OUT027,1787.636684
8,FDN33,OUT045,967.074500
9,FDA36,OUT017,2474.439647


In [68]:
sample_file['Item_Outlet_Sales'] = Y_pred
print(sample_file)
sample_file.to_csv('submission11.csv', index=False)

     Item_Identifier Outlet_Identifier  Item_Outlet_Sales
0              FDW58            OUT049        1611.163072
1              FDW14            OUT017        1233.186195
2              NCN55            OUT010         901.305281
3              FDQ58            OUT017        2447.000632
4              FDY38            OUT027        5726.511574
5              FDH56            OUT046        1871.509216
6              FDL48            OUT018         519.698592
7              FDC48            OUT027        2192.948117
8              FDN33            OUT045        1513.841396
9              FDA36            OUT017        3600.821452
10             FDT44            OUT017        1875.000501
11             FDQ56            OUT045        1285.436367
12             NCC54            OUT019         592.481550
13             FDU11            OUT049        2278.579280
14             DRL59            OUT013         884.715390
15             FDM24            OUT049        2343.124344
16            

In [69]:
'''
Changes:
1) Append a column called source to both train and test data
2) sum(x.isnull()
3) Extract categorical columns - data.dtypes[x]=='object' - except Item identifier, outlet identifier, source
4) Replace empty values in Outlet_Type with mode of Outlet_Type - mode(x).mode[0]
5) Item_Visibility - find mean. Should not be zero
6) Extract first two letters of Item_Identifier. Map Item_Type_Combined to Food, Non-Consumable, Drinks
7) Outlet_Years = 2013 - Outlet_Establishment_Year
8) Item_Fat_Content - replace LF and low fat as Low Fat, reg as Regular
9) Label Encode, OneHot Encode
10) data.drop['Item_Type','Outlet_Establishment_Year']
11) Separate test,train using source
12) test.drop['Item_Outlet_Sales','source'], train.drop['source']
13) to_csv
14) Find X_train,Y_train,X_test
15) Apply algo and determine Y_pred
'''

"\nChanges:\n1) Append a column called source to both train and test data\n2) sum(x.isnull()\n3) Extract categorical columns - data.dtypes[x]=='object' - except Item identifier, outlet identifier, source\n4) Replace empty values in Outlet_Type with mode of Outlet_Type - mode(x).mode[0]\n5) Item_Visibility - find mean. Should not be zero\n6) Extract first two letters of Item_Identifier. Map Item_Type_Combined to Food, Non-Consumable, Drinks\n7) Outlet_Years = 2013 - Outlet_Establishment_Year\n8) Item_Fat_Content - replace LF and low fat as Low Fat, reg as Regular\n9) Label Encode, OneHot Encode\n10) data.drop['Item_Type','Outlet_Establishment_Year']\n11) Separate test,train using source\n12) test.drop['Item_Outlet_Sales','source'], train.drop['source']\n13) to_csv\n14) Find X_train,Y_train,X_test\n15) Apply algo and determine Y_pred\n"